<a href="https://colab.research.google.com/github/bohoro/AutomaticRuleGeneration/blob/master/HateSpeechModelDesign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Solution Design - Create a Hate Speech Model 

## Problem Bounding


## E2E Architecture Design

## System Implementation

#### Prereqs
* Huggingface transformers and datasets

In [1]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 14.7 MB/s 
     |████████████████████████████████| 3.3 MB 64.8 MB/s 
     |████████████████████████████████| 895 kB 83.2 MB/s 
     |████████████████████████████████| 636 kB 87.3 MB/s 
     |████████████████████████████████| 52 kB 1.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 270 kB 10.4 MB/s 
     |████████████████████████████████| 123 kB 66.2 MB/s 
     |████████████████████████████████| 1.3 MB 71.1 MB/s 
     |████████████████████████████████| 243 kB 71.4 MB/s 
     |████████████████████████████████| 142 kB 70.0 MB/s 
     |████████████████████████████████| 294 kB 67.8 MB/s 


### Dataset

https://huggingface.co/datasets/hate_speech18


In [2]:
from datasets import load_dataset
dataset = load_dataset("hate_speech18")
dataset

Downloading:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/992 [00:00<?, ?B/s]

Using custom data configuration default


Downloading: 0.00B [00:00, ?B/s]

0 examples [00:00, ? examples/s]

Dataset hate_speech18 downloaded and prepared to /root/.cache/huggingface/datasets/hate_speech18/default/0.0.0/8033f254483a20d1d10b0f1b56ded1f54326b1617872c537f354a721a6951d47. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'user_id', 'subforum_id', 'num_contexts', 'label'],
        num_rows: 10944
    })
})

In [4]:
train = dataset['train']
print(type(train))
train[0]

<class 'datasets.arrow_dataset.Dataset'>


{'label': 0,
 'num_contexts': 0,
 'subforum_id': 1346,
 'text': 'As of March 13th , 2014 , the booklet had been downloaded over 18,300 times and counting .',
 'user_id': 572066}

## Utilize a pretrained tokenizer and foundation model (DistilBERT)

DistilBERT is a transformers model, smaller and faster than BERT, which was pretrained on the same corpus in a self-supervised fashion, using the BERT base model as a teacher. This means it was pretrained on the raw texts only, with no humans labelling them in any way (which is why it can use lots of publicly available data) with an automatic process to generate inputs and labels from those texts using the BERT base model.

***The model was trained on 8 16 GB V100 for 90 hours.***

@article{Sanh2019DistilBERTAD,
  title={DistilBERT, a distilled version of BERT: smaller, faster, cheaper and lighter},
  author={Victor Sanh and Lysandre Debut and Julien Chaumond and Thomas Wolf},
  journal={ArXiv},
  year={2019},
  volume={abs/1910.01108}
}

https://huggingface.co/distilbert-base-uncased

In [ ]:
foundation_model = "distilbert-base-uncased"
number_labels

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(foundation_model)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)